In [ ]:
import pandas as pd
import math
import numpy as np
import requests
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
BASE_URL = "https://api.weatherapi.com/v1/current.json"

# -----------------------------
# 1. Load Dataset and Preprocess
# -----------------------------
def parse_range(value):
    """Convert range string like '5–25' or '5-25' to average number."""
    if pd.isna(value):
        return None
    value = str(value).replace('–', '-')
    parts = value.split('-')
    if len(parts) == 2:
        try:
            return (float(parts[0].strip()) + float(parts[1].strip())) / 2
        except ValueError:
            return None
    else:
        try:
            return float(value.strip())
        except ValueError:
            return None

def load_historical_data(excel_file):
    df = pd.read_excel(excel_file)
    df.columns = (
        df.columns
        .str.strip()
        .str.replace('⁰', '°', regex=False)
        .str.replace(r'\s+', ' ', regex=True)
    )
    print("Columns after cleaning:", df.columns.tolist())

    # Convert Latitude and Longitude to float
    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

    # Clean climate columns
    climate_cols = [
        'Temp Winter (°C)', 'Temp Summer (°C)', 'Temp Monsoon (°C)',
        'Precip. Winter (mm)', 'Precip. Summer (mm)', 'Precip. Monsoon (mm)',
        'Wind Winter (km/hr)', 'Wind Summer (km/hr)', 'Wind Monsoon (km/hr)',
        'Humidity Winter (%)', 'Humidity Summer (%)', 'Humidity Monsoon (%)'
    ]
    for col in climate_cols:
        if col in df.columns:
            df[col] = df[col].apply(parse_range)
        else:
            print(f"Warning: Column '{col}' not found in dataset!")

    return df


# -----------------------------
# 2. Season Mapping
# -----------------------------
def month_to_season(month):
    if month in [11, 12, 1, 2]:
        return 'Winter'
    elif month in [6, 7, 8, 9]:
        return 'Monsoon'
    else:
        return 'Summer'

# -----------------------------
# 3. Geospatial Interpolation
# -----------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def inverse_distance_weighting(df, user_lat, user_lon, season, power=2):
    df['distance'] = df.apply(lambda row: haversine(user_lat, user_lon, row['Latitude'], row['Longitude']), axis=1)
    df = df[df['distance'] > 0]
    weights = 1 / (df['distance'] ** power)
    total_weight = weights.sum()
    if total_weight == 0:
        return None, None, None, None, "No nearby data points found."
    normalized_weights = weights / total_weight

    avg_temp_col = f'Temp {season} (°C)'
    avg_precip_col = f'Precip. {season} (mm)'
    avg_wind_col = f'Wind {season} (km/hr)'
    if season == 'Winter':
        avg_humidity_col = 'Humidity Winter (%)'
    elif season == 'Monsoon':
        avg_humidity_col = 'Humidity Monsoon (%)'
    else:
        avg_humidity_col = 'Humidity Summer (%)'

    avg_temp = np.average(df[avg_temp_col].dropna(), weights=normalized_weights[df[avg_temp_col].notna()])
    avg_precip = np.average(df[avg_precip_col].dropna(), weights=normalized_weights[df[avg_precip_col].notna()])
    avg_wind = np.average(df[avg_wind_col].dropna(), weights=normalized_weights[df[avg_wind_col].notna()])
    avg_humidity = np.average(df[avg_humidity_col].dropna(), weights=normalized_weights[df[avg_humidity_col].notna()])

    return avg_temp, avg_precip, avg_wind, avg_humidity, ""

# -----------------------------
# 4. Tourist Hazard Index
# -----------------------------
def calculate_thi(live_data, avg_data):
    thi_score = 0
    if avg_data['temp'] is not None and abs(live_data['temp'] - avg_data['temp']) > 10:
        thi_score += abs(live_data['temp'] - avg_data['temp']) / 10
    if avg_data['precip'] is not None and live_data['precip'] > avg_data['precip'] * 1.5:
        thi_score += live_data['precip'] / avg_data['precip']
    if avg_data['wind'] is not None and live_data['wind'] > avg_data['wind'] + 15:
        thi_score += live_data['wind'] / avg_data['wind']
    if live_data['humidity'] > 90:
        thi_score += live_data['humidity'] / 90

    if thi_score >= 5:
        return "Dangerous", thi_score
    elif thi_score >= 2:
        return "Risky", thi_score
    else:
        return "Normal", thi_score

# -----------------------------
# 5. API Integration
# -----------------------------
def fetch_live_data(location_name):
    if not WEATHER_API_KEY:
        print("Error: API key not found in environment variables.")
        return None
    try:
        params = {'key': WEATHER_API_KEY, 'q': location_name}
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        current = data['current']
        return {
            'temp': current['temp_c'],
            'precip': current['precip_mm'],
            'wind': current['wind_kph'],
            'humidity': current['humidity']
        }
    except Exception as e:
        print(f"Error fetching live data: {e}")
        return None

# -----------------------------
# 6. Main Monitoring
# -----------------------------
def monitor_tourist(df, user_lat, user_lon, month):
    # Find nearest city
    df['distance'] = df.apply(lambda row: haversine(user_lat, user_lon, row['Latitude'], row['Longitude']), axis=1)
    nearest_row = df.loc[df['distance'].idxmin()]
    city_name = nearest_row['Nearest City / Town']
    print(f"Nearest city/town from dataset: {city_name}")

    # Fetch live weather
    live_data = fetch_live_data(city_name)
    if live_data is None:
        print("Could not fetch live weather data.")
        return

    # Interpolated averages
    season = month_to_season(month)
    avg_temp, avg_precip, avg_wind, avg_humidity, _ = inverse_distance_weighting(df, user_lat, user_lon, season)
    avg_data = {'temp': avg_temp, 'precip': avg_precip, 'wind': avg_wind, 'humidity': avg_humidity}

    # THI calculation
    risk, thi_score = calculate_thi(live_data, avg_data)

    # Print results
    print(f"\n[{datetime.now()}] Risk Report:")
    print("-" * 50)
    print(f"Nearest City/Town: {city_name}")
    print(f"Season: {season}")
    print(f"Live Temperature (°C): {live_data['temp']}")
    print(f"Live Precipitation (mm): {live_data['precip']}")
    print(f"Live Wind Speed (km/hr): {live_data['wind']}")
    print(f"Live Humidity (%): {live_data['humidity']}")
    print(f"Overall Risk Level: {risk}")
    print(f"Tourist Hazard Index (THI) Score: {round(thi_score, 2)}")
    print("-" * 50)

# -----------------------------
# 7. Run in Colab
# -----------------------------
if __name__ == "__main__":
    df = load_historical_data('/content/sample_data/Northeast_India_Locations.xlsx')
    user_lat = float(input("Enter your latitude: "))
    user_lon = float(input("Enter your longitude: "))
    month = datetime.now().month
    monitor_tourist(df, user_lat, user_lon, month)


Columns after cleaning: ['Name', 'Nearest City / Town', 'State', 'Terrain', 'Latitude', 'Longitude', 'Temp Summer (°C)', 'Temp Monsoon (°C)', 'Precip. Winter (mm)', 'Precip. Summer (mm)', 'Precip. Monsoon (mm)', 'Wind Winter (km/hr)', 'Wind Monsoon (km/hr)', 'Wind Summer (km/hr)', 'Humidity Winter (%)', 'Humidity Monsoon (%)', 'Humidity Summer (%)']
Enter your latitude: 27.2
Enter your longitude: 92.4
Nearest city/town from dataset: Tawang

[2025-09-13 18:48:24.006333] Risk Report:
--------------------------------------------------
Nearest City/Town: Tawang
Season: Monsoon
Live Temperature (°C): 11.8
Live Precipitation (mm): 2.12
Live Wind Speed (km/hr): 3.6
Live Humidity (%): 97
Overall Risk Level: Normal
Tourist Hazard Index (THI) Score: 1.08
--------------------------------------------------


In [ ]:
%%writefile .env
WEATHER_API_KEY=3f6828e8f0534653a9f153444251309


Writing .env
